In [1]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [2]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [3]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [4]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163943
Total Vocab:  64


In [5]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163843


In [6]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

In [7]:
# normalize
X = X / float(n_vocab)

In [8]:
# one hot encode the output variable
y = to_categorical(dataY)

In [9]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
# define the checkpoint
#filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
#callbacks_list = [checkpoint]

In [11]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.keras"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [12]:
# fit the model
model.fit(X, y, epochs=10, batch_size=128, callbacks=callbacks_list)

Epoch 1/10
1281/1281 [==============================] - ETA: 0s - loss: 3.0127
Epoch 1: loss improved from inf to 3.01274, saving model to weights-improvement-01-3.0127.keras
1281/1281 [==============================] - 875s 674ms/step - loss: 3.0127
Epoch 2/10
1281/1281 [==============================] - ETA: 0s - loss: 2.8367
Epoch 2: loss improved from 3.01274 to 2.83672, saving model to weights-improvement-02-2.8367.keras
1281/1281 [==============================] - 886s 692ms/step - loss: 2.8367
Epoch 3/10
1281/1281 [==============================] - ETA: 0s - loss: 2.7550
Epoch 3: loss improved from 2.83672 to 2.75499, saving model to weights-improvement-03-2.7550.keras
1281/1281 [==============================] - 798s 623ms/step - loss: 2.7550
Epoch 4/10
1281/1281 [==============================] - ETA: 0s - loss: 2.6910
Epoch 4: loss improved from 2.75499 to 2.69105, saving model to weights-improvement-04-2.6910.keras
1281/1281 [==============================] - 905s 706ms/step

In [13]:
#import sys
# load the network weights
#filename = "weights-improvement-20-2.1411.hdf5"
#model.load_weights(filename)
#model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
import sys
# load the network weights
filename = "weights-improvement-10-2.4076.keras"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [18]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([chr(value) for value in pattern]), "\"")

Seed:
" /#"")&+$ )&11)"!&##"/"+121&#&;*+,11%"0*"
1%"+"51.2"01&,+&0
4%, &+1%"4,/)!*& "


In [19]:
# generate characters
for i in range(1000):
 x = np.reshape(pattern, (1, len(pattern), 1))
 x = x / float(n_vocab)
 prediction = model.predict(x, verbose=0)
 index = np.argmax(prediction)
 result = int_to_char[index]
 seq_in = [chr(value) for value in pattern]
 sys.stdout.write(result)
 pattern.append(index)
 pattern = pattern[1:len(pattern)]
print("\nDone.")

NameError: name 'int_to_char' is not defined